In [0]:
import pandas as pd
import numpy as np

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c mais202-fall2019

 43% 9.00M/20.7M [00:00<00:00, 36.9MB/s]
100% 20.7M/20.7M [00:00<00:00, 59.6MB/s]
 68% 35.0M/51.7M [00:00<00:00, 27.5MB/s]
100% 51.7M/51.7M [00:01<00:00, 50.9MB/s]
  0% 0.00/429k [00:00<?, ?B/s]
100% 429k/429k [00:00<00:00, 131MB/s]
  0% 0.00/127 [00:00<?, ?B/s]
100% 127/127 [00:00<00:00, 118kB/s]
  0% 0.00/165k [00:00<?, ?B/s]
100% 165k/165k [00:00<00:00, 128MB/s]


In [0]:
!unzip /content/train_images.npy.zip
!unzip /content/test_images.npy.zip

Archive:  /content/train_images.npy.zip
replace train_images.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  /content/test_images.npy.zip
replace test_images.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
train_images = np.load("train_images.npy")
test_images = np.load("test_images.npy")

train_labels = np.loadtxt(open("train_labels.csv", "rb"), delimiter=",", skiprows=1)
train_labels = train_labels[:, 1]

train_images, test_images = train_images / 255.0, test_images / 255.0

train_images = train_images.reshape(*train_images.shape, 1)
test_images = test_images.reshape(*test_images.shape, 1)

from sklearn.model_selection import train_test_split
train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=456)


In [0]:
train_images.shape

(45000, 28, 28, 1)

In [0]:
# import tensorflow

from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
# define keras model

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu',padding='same',input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# display model summary and compile it
model.summary()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 12, 12, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

In [0]:
model.fit(train_images, train_labels,  epochs=5, validation_data=(valid_images, valid_labels))

Train on 45000 samples, validate on 5000 samples
Epoch 1/5
45000/45000 [==============================] - 20s 445us/sample - loss: 0.6480 - accuracy: 0.7604 - val_loss: 0.4837 - val_accuracy: 0.8136
Epoch 2/5
45000/45000 [==============================] - 14s 321us/sample - loss: 0.4641 - accuracy: 0.8289 - val_loss: 0.4804 - val_accuracy: 0.8284
Epoch 3/5
23328/45000 [==============>...............] - ETA: 6s - loss: 0.4245 - accuracy: 0.8431

KeyboardInterrupt: ignored

In [0]:
# for generating submission

y_test = model.predict(test_images)
y_test = np.argmax(y_test, axis=1)
print(y_test.shape)

df_test = pd.read_csv('sample_submission.csv')
df_test['label'] = y_test
df_test.to_csv('submission.csv', index=False)